In [1]:
import math
from tensorboardX import SummaryWriter
import roboschool
import quadruppedEnv
import env_configurations
sess = None

robot lib not fouund

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers

env_name = 'Pendulum-v0'#'QuadruppedWalk-v1' #'QuadruppedWalk-v1' #$'BipedalWalker-v2'#'RoboschoolAnt-v1'
#'BipedalWalker-v2'#'RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' 
#'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)
print(action_space.low)
print(action_space.high)

[-2.]
[2.]


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-06-04 18:06:28,593	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-04_18-06-28_12791/logs.
2019-06-04 18:06:28,713	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:63749 to respond...
2019-06-04 18:06:28,839	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:32657 to respond...
2019-06-04 18:06:28,842	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-06-04 18:06:28,898	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-04_18-06-28_12791/logs.
2019-06-04 18:06:28,900	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.104',
 'redis_address': '192.168.3.104:63749',
 'object_store_address': '/tmp/ray/session_2019-06-04_18-06-28_12791/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-06-04_18-06-28_12791/sockets/raylet',
 'webui_url': None}

In [5]:
from a2c_continuous import A2CAgent
import tr_helpers
import networks
import a2c_games_configurations

#a2c_config = a2c_games_configurations.bipedalwalkerhardcore_config
#a2c_config = a2c_games_configurations.roboschoolant_config
a2c_config = a2c_games_configurations.pendulum_config

agent = A2CAgent(sess,'car', obs_space, False, action_space, a2c_config)
#agent.restore('nn/latest')
agent.train()


Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


2019-06-04 18:06:30,188	ERROR worker.py:1672 -- WARNING: 24 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=12873) 
(pid=12873) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=12873) For more information, please see:
(pid=12873)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=12873)   * https://github.com/tensorflow/addons
(pid=12873) If you depend on functionality not listed there, please file an issue.
(pid=12873) 
(pid=12876) 
(pid=12876) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=12876) For more information, please see:
(pid=12876)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=12876)   * https://github.com/tensorflow/addons
(pid=12876) If you depend on functionality not listed there, please file an issue.
(pid=12876) 
(pid=12876) robot lib not fouund
(pid=12876) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  C

Frames per seconds:  914.0110763238046
Frames per seconds:  2495.402669839735
Frames per seconds:  2480.816010424705
Frames per seconds:  2455.9061682311394
Frames per seconds:  2369.6057519790083
Frames per seconds:  2456.5972842810893
Frames per seconds:  2373.8019233745276
Frames per seconds:  2392.030493469345
Frames per seconds:  2427.5061979594097
Frames per seconds:  2273.460649510686
Frames per seconds:  2400.5710529728185
Frames per seconds:  2373.2824907333525
Frames per seconds:  2322.537980679798
saving next best rewards:  -1361.4993422160762
Frames per seconds:  2396.5418647525435
Frames per seconds:  2412.4421717200084
Frames per seconds:  2513.4464827867105
Frames per seconds:  2422.609746467306
Frames per seconds:  2441.5486505722192
Frames per seconds:  2294.839908184332
Frames per seconds:  2395.739787767633
Frames per seconds:  2340.056235875106
Frames per seconds:  2431.976227039025
Frames per seconds:  2383.190745492156
Frames per seconds:  2363.5863477772764
Frame

Frames per seconds:  2130.731834318587
Frames per seconds:  2229.655366891416
Frames per seconds:  2203.2523720411045
Frames per seconds:  2246.420499645382
Frames per seconds:  1502.9686204357697
Frames per seconds:  1927.2976057351789
Frames per seconds:  2188.366714969052
Frames per seconds:  2280.7574275196853
Frames per seconds:  2145.9128398012654
Frames per seconds:  2269.496308539712
Frames per seconds:  2207.2561443900836
Frames per seconds:  2254.533403043302
Frames per seconds:  2252.0319806706543
Frames per seconds:  2129.700647592602
Frames per seconds:  2217.22879703968
Frames per seconds:  2224.357441166722
Frames per seconds:  2226.626873595063
Frames per seconds:  2139.3796777805674
Frames per seconds:  2229.798904353099
Frames per seconds:  2239.467537641069
Frames per seconds:  2183.405027512638
Frames per seconds:  2217.347843864482
Frames per seconds:  2208.1049616162113
Frames per seconds:  2205.0984605779813
Frames per seconds:  2257.823481543007
Frames per secon

...

Frames per seconds:  1621.3346218543227
Frames per seconds:  1834.3116243711188
Frames per seconds:  1925.8284918966617
Frames per seconds:  1834.0171113604881
Frames per seconds:  1768.3349511285317
Frames per seconds:  2146.6207064759965
Frames per seconds:  1952.8188534160722
Frames per seconds:  1936.9454277818265
Frames per seconds:  2159.9769547060196
Frames per seconds:  2114.6099221502523
Frames per seconds:  2364.1223650430884
Frames per seconds:  2338.3643281030522
Frames per seconds:  2380.8920158852993
Frames per seconds:  2371.2751075501533
Frames per seconds:  2019.5644365866044
Frames per seconds:  2281.198371759027
Frames per seconds:  2188.6477345922103
Frames per seconds:  2191.6542477083135
Frames per seconds:  2414.758812390646
Frames per seconds:  2428.434038819054
Frames per seconds:  2331.474271619092
Frames per seconds:  2087.911361046618
Frames per seconds:  2230.1045618057806
Frames per seconds:  2348.0185175061283
Frames per seconds:  2180.5406441655496
Frame

Frames per seconds:  2452.5964577270797
Frames per seconds:  2390.055122491953
Frames per seconds:  2432.675984349193
Frames per seconds:  2437.7903495687433
Frames per seconds:  2216.7527375427353
Frames per seconds:  2285.305874690856
Frames per seconds:  2193.2838003182455
Frames per seconds:  2438.7039146748066
Frames per seconds:  2077.999500695736
Frames per seconds:  1477.7904115961496
Frames per seconds:  2030.3065365495777
Frames per seconds:  2178.182958618773
Frames per seconds:  1646.3811905554116
Frames per seconds:  1673.0631647506475
Frames per seconds:  1633.211483294344
Frames per seconds:  2380.8920158852993
Frames per seconds:  2395.4297952463603
Frames per seconds:  1770.3642056424553
Frames per seconds:  2340.290849965236
Frames per seconds:  2353.9587892393392
Frames per seconds:  2319.3573014679832
Frames per seconds:  2409.4160899731623
Frames per seconds:  2371.0551833482755
Frames per seconds:  2024.4764583883255
Frames per seconds:  2465.255020089542
Frames p

Frames per seconds:  2336.0139933470687
Frames per seconds:  2467.447425447476
Frames per seconds:  2451.420576793118
Frames per seconds:  2493.3397980698674
Frames per seconds:  2459.4163414142527
Frames per seconds:  2393.1447420621653
Frames per seconds:  2109.5621586363527
Frames per seconds:  1722.4570389073724
Frames per seconds:  2338.2319325974336
Frames per seconds:  2375.330057804747
Frames per seconds:  2427.456806199868
Frames per seconds:  2439.307796984436
Frames per seconds:  2412.387971107292
Frames per seconds:  2100.717475559103
Frames per seconds:  1980.97113802239
Frames per seconds:  2320.7408790676764
Frames per seconds:  2421.1675900433165
Frames per seconds:  2267.713270157194
Frames per seconds:  2321.0920488886777
Frames per seconds:  1958.9360529076396
Frames per seconds:  2025.8706338499853
Frames per seconds:  1901.0639382023367
Frames per seconds:  1950.0454466205615
Frames per seconds:  2121.772014725556
Frames per seconds:  1742.7987963032094
Frames per 

Exception in thread ray_listen_error_messages:
Traceback (most recent call last):
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/redis/connection.py", line 185, in _read_from_socket
    raise socket.error(SERVER_CLOSED_CONNECTION_ERROR)
OSError: Connection closed by server.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/ray/worker.py", line 1650, in listen_error_messages_raylet
    msg = worker.error_message_pubsub_client.get_message()
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/redis/client.py", line 3135, in get_message
    response = self.parse_response(block=False, timeout=time

KeyboardInterrupt: 

In [6]:
ray.shutdown()

In [9]:
import numpy as np
#agent.restore('nn/latest')
#agent.restore('nn/rob_20QuadruppedWalk-v1')
#agent.restore('nn/ppo_continuousQuadruppedWalk-v1')
env = env_configurations.a2c_configurations[env_name]['ENV_CREATOR']()
def evaluate(env,t_max=5000):
    rewards = []
    #env._max_episode_steps = 5000
    env.batch_mode = False
    print('reset')
    #env = env.old_env
    s = env.reset()
    reward = 0
    for it in range(5000):
        action = agent.get_action([s], False)
        
        #ad = agent.get_action_distribution([s])
        #print(ad)
        s, r, done, _ = env.step(np.squeeze(action, axis = 0))
        print(action)
        env.render()
        reward += r
        
            
        if done:
            break       
        
    return reward

import gym.wrappers

#env_monitor = wrappers.make_atari_deepmind(env_name, noop_max=30, skip=4)
#env_monitor = wrappers.ReallyDoneWrapper(env_monitor)
#env_monitor = gym.wrappers.Monitor(env,directory='video_ppo',force=True)

sessions = [print('reward:', evaluate(env)) for _ in range(1)]
#env_monitor.close()




reset
[[1.3137766]]
[[1.892915]]
[[1.3765378]]
[[1.3291712]]
[[1.2196888]]
[[0.600177]]
[[1.6781496]]
[[1.1732856]]
[[1.3007128]]
[[0.56357914]]
[[1.1586227]]
[[2.]]
[[1.3055363]]
[[1.1068195]]
[[1.7129254]]
[[0.4191435]]
[[0.03353286]]
[[0.6783081]]
[[1.8611296]]
[[2.]]
[[1.6170692]]
[[2.]]
[[1.0799958]]
[[-0.782838]]
[[0.9267972]]
[[0.95904905]]
[[0.36568236]]
[[1.5928519]]
[[2.]]
[[2.]]
[[1.6052634]]
[[0.93034106]]
[[2.]]
[[2.]]
[[2.]]
[[2.]]
[[-0.41032302]]
[[2.]]
[[-1.3916272]]
[[-0.07630211]]
[[-1.2611693]]
[[-0.72901875]]
[[-2.]]
[[-1.2073874]]
[[-0.89806074]]
[[0.5516576]]
[[0.82253766]]
[[1.9160943]]
[[-0.39691997]]
[[1.620208]]
[[1.169594]]
[[0.6419371]]
[[1.5052397]]
[[1.7755823]]
[[0.96587634]]
[[1.0225468]]
[[2.]]
[[0.11486804]]
[[0.8908909]]
[[0.8643489]]
[[1.6572382]]
[[0.82482874]]
[[2.]]
[[1.3868709]]
[[1.8087618]]
[[1.1245999]]
[[-0.02638596]]
[[1.2672324]]
[[-0.05156064]]
[[1.4959651]]
[[0.64511675]]
[[0.8481275]]
[[1.9731072]]
[[0.37995815]]
[[1.646808]]
[[2.]]
[[2.

In [ ]:
obs = env.reset()
obs, r, done, _ = env.step(2)

done = False
for _ in range(130):
    obs, _, done, _ = env.step(1)
    #env.render()
    obs = np.array(obs)
    if done:
        print('done')
        break
plt.title("what your network gonna see")
print (obs.shape)
plt.imshow(obs[:,:,0],interpolation='none',cmap='gray');

In [ ]:
import gym
gym.envs.registry.all()